In [23]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix,classification_report,roc_auc_score
from sklearn.metrics import roc_auc_score, log_loss, precision_recall_curve, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.pipeline import Pipeline
import requests
import io
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
sns.set_theme(style="dark")
pd.set_option('display.max_columns', 0)
plt.style.use('ggplot')
pd.options.display.float_format = '{:.2f}'.format
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [24]:
df = pd.read_csv('./Ad_Click_prediction_train.csv')

In [25]:
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463291 entries, 0 to 463290
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   session_id              463291 non-null  int64         
 1   DateTime                463291 non-null  datetime64[ns]
 2   user_id                 463291 non-null  int64         
 3   product                 463291 non-null  object        
 4   campaign_id             463291 non-null  int64         
 5   webpage_id              463291 non-null  int64         
 6   product_category_1      463291 non-null  int64         
 7   product_category_2      97437 non-null   float64       
 8   user_group_id           445048 non-null  float64       
 9   gender                  445048 non-null  object        
 10  age_level               445048 non-null  float64       
 11  user_depth              445048 non-null  float64       
 12  city_development_index  338162

In [27]:
maximum = df['DateTime'].max()
maximum - df.groupby('user_id')['DateTime'].max()

user_id
4         5 days 09:08:00
6         1 days 11:26:00
19        2 days 14:37:00
25        4 days 03:45:00
26        1 days 03:59:00
                ...      
1141714   0 days 20:08:00
1141716   0 days 06:02:00
1141718   1 days 09:43:00
1141723   2 days 05:21:00
1141729   2 days 03:15:00
Name: DateTime, Length: 150347, dtype: timedelta64[ns]

In [28]:
df = df.sort_values(['user_id', 'DateTime'])
df['last_click_time'] = df.groupby('user_id')['DateTime'].shift(1)
df['time_since_last_click_sec'] = (df['DateTime'] - df['last_click_time']).dt.total_seconds().fillna(-1)
df.tail()

,session_id,DateTime,user_id,product,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click,last_click_time,time_since_last_click_sec
291898,566299,2017-07-05 18:38:00,1141723,H,118601,28529,5,82527.00,1.00,Male,1.00,3.00,4.00,1,0,2017-07-05 18:33:00,300.00
281609,15398,2017-07-05 16:31:00,1141729,H,82320,1734,1,NaN,1.00,Male,1.00,3.00,NaN,0,0,NaT,-1.00
281610,124110,2017-07-05 16:31:00,1141729,C,359520,13787,4,NaN,1.00,Male,1.00,3.00,NaN,0,0,2017-07-05 16:31:00,0.00
307180,22020,2017-07-05 20:44:00,1141729,C,360936,13787,5,NaN,1.00,Male,1.00,3.00,NaN,0,0,2017-07-05 16:31:00,15180.00
307181,296274,2017-07-05 20:44:00,1141729,C,360936,13787,5,NaN,1.00,Male,1.00,3.00,NaN,0,0,2017-07-05 20:44:00,0.00


In [20]:
df['user_product_count'] = df.groupby(['user_id', 'product']).cumcount()
df['user_cat_1_count'] = df.groupby(['user_id', 'product_category_1']).cumcount()
df['user_cat_2_count'] = df.groupby(['user_id', 'product_category_2']).cumcount()

In [31]:
df.session_id.nunique(),len(df)

(463291, 463291)

In [21]:
df.head()

,session_id,DateTime,user_id,product,campaign_id,webpage_id,product_category_1,product_category_2,user_group_id,gender,age_level,user_depth,city_development_index,var_1,is_click,last_click_time,time_since_last_click_sec,user_cat_count,user_product_count,user_cat_1_count,user_cat_2_count
45068,4321,2017-07-02 14:51:00,4,I,404347,53587,1,146115.00,11.00,Female,5.00,2.00,NaN,0,0,NaT,-1.00,0,0,0,0.00
347168,106452,2017-07-06 12:33:00,6,C,405490,60305,3,NaN,3.00,Male,3.00,3.00,3.00,1,0,NaT,-1.00,0,0,0,NaN
47703,538078,2017-07-02 15:25:00,19,E,98970,6970,2,NaN,0.00,Male,0.00,3.00,2.00,1,0,NaT,-1.00,0,0,0,NaN
250894,538079,2017-07-05 09:22:00,19,E,98970,6970,2,NaN,0.00,Male,0.00,3.00,2.00,1,0,2017-07-02 15:25:00,237420.00,1,1,1,NaN
150872,354959,2017-07-03 20:14:00,25,B,360936,13787,2,NaN,2.00,Male,2.00,3.00,4.00,1,0,NaT,-1.00,0,0,0,NaN


In [ ]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from lightgbm import LGBMClassifier
from scipy.stats import randint, uniform

def tune_lightgbm_robust(X, y):
    """
    Performs Randomized Search with Stratified K-Fold CV.
    Addresses Evaluator Q2: "Rigorous hyperparameter tuning and cross-validation"
    """
    
    # 1. Define the Estimator
    # Note: We use scale_pos_weight to handle imbalance instead of SMOTE
    # (Typical ratio is sum(negative) / sum(positive))
    ratio = float(np.sum(y == 0)) / np.sum(y == 1)
    
    lgbm = LGBMClassifier(
        objective='binary',
        metric='auc',
        n_jobs=-1,
        random_state=42,
        scale_pos_weight=ratio, # Automatically handle imbalance
        verbosity=-1
    )

    # 2. Define the Hyperparameter Search Space
    # We search over a range of values to find the best combination
    param_dist = {
        'n_estimators': randint(100, 1000),      # Number of trees
        'learning_rate': uniform(0.01, 0.2),     # Speed of learning
        'max_depth': randint(3, 10),             # Depth of tree (prevent overfitting)
        'num_leaves': randint(20, 100),          # Complexity of tree
        'subsample': uniform(0.6, 0.4),          # Train on x% of rows
        'colsample_bytree': uniform(0.6, 0.4),   # Train on x% of features
        'reg_alpha': uniform(0, 10),             # L1 Regularization
        'reg_lambda': uniform(0, 10)             # L2 Regularization
    }

    # 3. Setup Stratified K-Fold
    # Splits data into 5 chunks, ensuring each chunk has the same % of clicks
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # 4. Setup RandomizedSearchCV
    # n_iter=50 means it will try 50 different random combinations
    search = RandomizedSearchCV(
        estimator=lgbm,
        param_distributions=param_dist,
        n_iter=50,
        scoring='roc_auc',
        cv=cv,
        verbose=1,
        random_state=42,
        n_jobs=-1
    )

    # 5. Execute Tuning
    print("Starting Hyperparameter Tuning... this may take a few minutes.")
    search.fit(X, y)
    
    print(f"Best AUC Score: {search.best_score_:.4f}")
    print("Best Parameters:")
    print(search.best_params_)
    
    return search.best_estimator_

# --- Usage ---
# Assuming you have your X_train and y_train ready from the previous steps
# best_model = tune_lightgbm_robust(X_train, y_train)

In [ ]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from scipy.stats import randint, uniform, loguniform
import numpy as np
import time

# ---------------------------------------------------------
# 1. The Generic Tuning Function
# ---------------------------------------------------------
def tune_model_robust(model_name, estimator, param_dist, X, y, n_iter=20):
    """
    Universally tunes any model using Stratified K-Fold CV.
    """
    print(f"\n{'='*40}")
    print(f"Stats: Tuning {model_name}...")
    print(f"{'='*40}")
    
    start_time = time.time()
    
    # Stratified K-Fold for stable validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Randomized Search
    search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_dist,
        n_iter=n_iter,       # Tries 'n_iter' random combinations
        scoring='roc_auc',   # Optimize for AUC
        cv=cv,
        verbose=1,
        random_state=42,
        n_jobs=-1            # Use all CPU cores
    )

    search.fit(X, y)
    
    elapsed = time.time() - start_time
    print(f"Done! Time taken: {elapsed/60:.2f} min")
    print(f"Best AUC: {search.best_score_:.4f}")
    print(f"Best Params: {search.best_params_}")
    
    return search.best_estimator_

# ---------------------------------------------------------
# 2. Configuration: Define Models & Grids
# ---------------------------------------------------------
# Calculate imbalance ratio for Boosting models
# ratio = negative / positive
ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

model_configs = {
    # --- Tree Ensembles (The most powerful for this data) ---
    "LightGBM": {
        "model": LGBMClassifier(objective='binary', scale_pos_weight=ratio, verbosity=-1, random_state=42),
        "params": {
            'n_estimators': randint(100, 1000),
            'learning_rate': uniform(0.01, 0.2),
            'max_depth': randint(3, 10),
            'num_leaves': randint(20, 100),
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4),
            'reg_alpha': uniform(0, 10),
            'reg_lambda': uniform(0, 10)
        }
    },
    
    "XGBoost": {
        "model": XGBClassifier(eval_metric='logloss', scale_pos_weight=ratio, use_label_encoder=False, random_state=42),
        "params": {
            'n_estimators': randint(100, 1000),
            'learning_rate': uniform(0.01, 0.2),
            'max_depth': randint(3, 10),
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4),
            'gamma': uniform(0, 5)
        }
    },
    
    "RandomForest": {
        "model": RandomForestClassifier(class_weight='balanced', random_state=42),
        "params": {
            'n_estimators': randint(100, 500),
            'max_depth': randint(5, 20),
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10),
            'max_features': ['sqrt', 'log2']
        }
    },

    # --- Linear Models ---
    "LogisticRegression": {
        "model": LogisticRegression(class_weight='balanced', solver='saga', max_iter=500, random_state=42),
        "params": {
            'C': loguniform(1e-4, 10),  # Log-uniform searches 0.0001 to 10 efficiently
            'penalty': ['l1', 'l2']
        }
    },

    # --- Simple Trees ---
    "DecisionTree": {
        "model": DecisionTreeClassifier(class_weight='balanced', random_state=42),
        "params": {
            'max_depth': randint(3, 20),
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 20),
            'criterion': ['gini', 'entropy']
        }
    },
    
    # --- Others (Warning: KNN/SVC can be very slow on large data) ---
    "AdaBoost": {
        "model": AdaBoostClassifier(random_state=42),
        "params": {
            'n_estimators': randint(50, 500),
            'learning_rate': uniform(0.01, 1.0)
        }
    }
}

In [ ]:
def run_comprehensive_tuning(df_label_encoded, df_one_hot_scaled, target_col):
    """
    Handles different X_train/X_val for different model types.
    df_label_encoded: The dataframe processed for Tree models.
    df_one_hot_scaled: The dataframe processed for Linear models.
    """
    results = {}
    
    # Define Target
    y = df_label_encoded[target_col] 
    
    # 1. Define Model Groups
    tree_models = ['LightGBM', 'XGBoost', 'RandomForest']
    linear_models = ['LogisticRegression'] # Add KNN here if needed
    # Select which models to run (You can comment out slow ones like SVC)
    models_to_run = ['LightGBM', 'XGBoost', 'RandomForest', 'LogisticRegression']
    
    # 2. Iterate through the configurations
    for name in model_configs.keys():
        # Skip if not in our execution list
        if name not in models_to_run:
            continue
            
        # Select the correct data source
        if name in tree_models:
            X = df_label_encoded.drop(columns=[target_col])
            print(f"\n>>> Using Label-Encoded data for {name}")
        elif name in linear_models:
            X = df_one_hot_scaled.drop(columns=[target_col])
            print(f"\n>>> Using One-Hot Scaled data for {name}")
        else:
            X = df_label_encoded.drop(columns=[target_col])
            
        # Call the tuning engine (from our previous step)
        config = model_configs[name]
        best_model = tune_model_robust(
            model_name=name,
            estimator=config["model"],
            param_dist=config["params"],
            X=X,
            y=y,
            n_iter=15 
        )
        
        results[name] = best_model
        
    return results

# --- Usage ---
# tuned_models = run_comprehensive_tuning(df_trees, df_linear, 'is_click')